<strong>PyChemkin</strong> has many <em>Mixture</em> methods facilitate the evaluations of commonly used <u>mixture-averaged</u> thermodynamic and transport properties as well as species rate of production (ROP), chemical heat release rate (HRR), and the forwards and reverse rates of a reaction. This tutorial will show you how to use some of these <em>Mixture</em> utility methods.

To start a <strong>PyChemkin</strong> project, you always have to import these Python packages: <code>os</code>, <code>chemkin</code>, and <code>numppy</code>. The <code>matplotlib</code> package is also imported for plotting purposes.  

In [ ]:
import os
import chemkin as ck                      # Chemkin
import numpy as np                        # number crunching
%matplotlib inline
import matplotlib.pyplot as plt           # plotting

# check working directory
current_dir = os.getcwd()
print("current working directory: " + current_dir)
# set verbose mode
ck.setverbose(True)

<h5>Set up the <em>Chemistry Set</em></h5>The C2 combustion mechanism, <i><u>C2_NOx_SRK.inp</u></i> is used to create <em>Chemistry Set</em> <code>MyGasMech</code>. The mechanism input file contains all the required thermodynamic and transport data already. Call the <code>preprocesstransportdata</code> method before preprocessing the <em>Chemistry Set</em> so that you will be able obtain the mixture transport properties as well. 

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = os.path.join(ck.ansys_dir, "reaction", "data")
mechanism_dir = data_dir
# create a chemistry set based on the C2 NOx mechanism
MyGasMech = ck.Chemistry(label="C2 NOx")
# set mechanism input files
# inclusion of the full file path is recommended
MyGasMech.chemfile = os.path.join(mechanism_dir, "C2_NOx_SRK.inp")
# this mechanism file contains all the necessary thermodynamic and transport data
# therefore no need to specify the therm and the tran data files
# instruct the preprocessor to include the transport properties (when the tran data file is not provided)
MyGasMech.preprocesstransportdata()
# preprocess the mechanism files
iError = MyGasMech.preprocess()

<h5>Create a <em>Mixture</em></h5>Instantiate the <code>premixed</code> <em>Mixture</em> and set it up using the <i><u>recipe</u></i> method. 

In [ ]:
# create a mixture called premixed based on the MyGasMech chemistry set
premixed = ck.Mixture(MyGasMech)
# set mixture pressure [dynes/cm2]
mixpressure = 2.0                                # given in atm
# convert to dynes/cm2
premixed.pressure = mixpressure * ck.Patm
# set mixture temperature [K]
premixed.temperature = 500.0
# molar composition of the mixture
mixture_recipe = [("CH4", 0.08), ("N2", 0.6), ("O2", 0.2), ("H2O", 0.12)]
# set mixture mole fractions
premixed.X = mixture_recipe

<h5>Mixture-averaged properties</h5>

Now you are ready to compute some mixture properties and plot them as functions of both temperature and pressure.<p><u>Mixture density</u> is one of the most frequently required values in reactor simulations. Once the mixture <coed>premixed</code> is properly set up, you can call the method <code>RHO</code> to get the density of the mixture. Likewise, use methods <code>HML</code> and <code>mixtureviscocity</code> to obtain the molar-based enthalpy and the mixture-averaged viscosity of mixture <code>premixed</code>, respectively. The mixture-averaged diffusion coefficients of <u>all</u> species are computed by the method <code>mixturediffusioncoeffs</code>  and are stored in the 1-D array <code>diffcoeffs</code>. The mixture-averaged species diffusion coefficient represents the <i>apparent diffusion coefficient</i> of the underlying species against the mixture. Here the mixture-averaged diffusion coefficient of methane (CH4) is plotted.   

In [ ]:
#
# test mixture properties
#
plt.subplots(2, 2, sharex="col", figsize=(9, 9))
dTemp = 20.0
points = 100
# curve attributes
curvelist = ["g", "b--", "r:"]
# list of pressure values for the plot
press = [1.0, 5.0, 10.0]                         # given in atm
# create arrays for the plot
# mixture density
rho = np.zeros(points, dtype=np.double)
# mixture enthalpy
enthalpy = np.zeros_like(rho, dtype=np.double)
# mixture viscosity
visc = np.zeros_like(rho, dtype=np.double)
# mixture averaged diffusion coefficient of CH4
diff_CH4 = np.zeros_like(rho, dtype=np.double)
# species index of CH4
CH4_index = MyGasMech.getspecindex("CH4")
# temperature data
T = np.zeros_like(rho, dtype=np.double)
# start of the plotting loop #1
k = 0
# loop over the pressure values
for j in range(len(press)):
    # starting temperature at 300K
    temp = 300.0
    # loop over temperature data points
    for i in range(points):
        # set mixture pressure [dynes/cm2]
        premixed.pressure = press[j] * ck.Patm
        # set mixture temperature [K]
        premixed.temperature = temp
        # get mixture density [gm/cm3]
        rho[i] = premixed.RHO
        # get mixture enthalpy [ergs/mol] and convert it to [kJ/mol]
        enthalpy[i] = premixed.HML() * 1.0e-3 / ck.ergsperjoule
        # get mixture viscosity [gm/cm-sec]
        visc[i] = premixed.mixtureviscosity()
        # get mixture-averaged diffusion coefficient of CH4 [cm2/sec]
        diffcoeffs = premixed.mixturediffusioncoeffs()
        diff_CH4[i] = diffcoeffs[CH4_index]
        T[i] = temp
        temp += dTemp
    # create sub plots
    # plot mixture density versus temperature
    plt.subplot(221)
    plt.plot(T, rho, curvelist[k])
    plt.ylabel("Density [g/cm3]")
    plt.legend(("1 atm", "5 atm", "10 atm"), loc="upper right")
    # plot mixture enthalpy versus temperature
    plt.subplot(222)
    plt.plot(T, enthalpy, curvelist[k])
    plt.ylabel("Enthalpy [kJ/mole]")
    plt.legend(("1 atm", "5 atm", "10 atm"), loc="upper left")
    # plot mixture viscosity versus temperature
    plt.subplot(223)
    plt.plot(T, visc, curvelist[k])
    plt.xlabel("Temperature [K]")
    plt.ylabel("Viscosity [g/cm-sec]")
    plt.legend(("1 atm", "5 atm", "10 atm"), loc="lower right")
    # plot mixture averaged CH4 diffusion coefficient versus temperature
    plt.subplot(224)
    plt.plot(T, diff_CH4, curvelist[k])
    plt.xlabel("Temperature [K]")
    plt.ylabel(r"$D_{CH_4}$ [cm2/sec]")
    plt.legend(("1 atm", "5 atm", "10 atm"), loc="upper left")
    k += 1
# plot legends
plt.legend(("1 atm", "5 atm", "10 atm"), loc="upper left")
# display the plots
plt.show()

Note that, for ideal gas, the enthalpy does <u>not</u> depend on the pressure.

In [ ]:
# delete unused objects
del enthalpy, visc, diff_CH4, diffcoeffs

<h5>Real-gas mixture properties</h5>

When you preprocess the <code>MyGasMech</code>, the <strong>PyChemkin</strong> output indicates that the <i><u>Soave</u> cubic Equation of State (EOS) real-gas model</i> is available from this <i><u>C2_NOx_SRK.inp</u></i> mechanism. You can turn on this real-gas model to re-calculate the mixture density and compare it with the ideal gas density. The <em>Chemkin</em> real-gas cubic EOS model is enabled by the <code>EOS_</code> data block in the mechanism input file.<br><hr>Please visit the on-line <a href = 'https://ansyshelp.ansys.com/account/secured?returnurl=/Views/Secured/prod_page.html?pn=Chemkin&pid=ChemkinPro&lang=en'> <em>Chemkin</em> document portal </a> for detail information about the <em>Chemkin</em> real-gas models.

Use a relatively large pressure value to make the difference between the <u>ideal gas</u> and the <u>real-gas</u> density values more visible. 

In [ ]:
# set mixture pressure [dynes/cm2]
premixed.pressure = 200.0 * ck.Patm


Switch on the <em>Chemkin</em> real-gas EOS model by using the <em>Mixture</em>method <code>userealgascubicEOS</code>. You can use the <code>setrealgasmixingrule</code> method to select one of the two available mixng rules when calculating real-gas mixture properties. The default mixing rule is the <i><u>Van der Waals</u></i> mixing rule (<code>rule=0</code>).

In [ ]:
# turn on real-gas cubic equation of state
premixed.userealgascubicEOS()
# set mixture mixing rule to Van der Waals (default)
premixed.setrealgasmixingrule(rule=0)

<hr>You can use the <code>help</code> method to get hints on the <strong>PyChemkin</strong> real gas model options. For example,

In [ ]:
ck.help(topic="real gas")

Create array <code>rho_real</code> to hold the real-gas density values.

In [ ]:
# real-gas mixture density
rho_real = np.zeros(points, dtype=np.double)

Use the same <em>Mixture</em> method <code>RHO</code> to compute the <u>real-gas</u> density. 

In [ ]:
# loop over temperature data points
for i in range(len(T)):
    # set mixture temperature [K]
    premixed.temperature = T[i]
    # get mixture density [gm/cm3]
    rho_real[i] = premixed.RHO


Switch back to use the <u>ideal gas law</u> by calling the <code>useidealgaslaw</code> method.

In [ ]:
# turn ON the ideal gas law
premixed.useidealgaslaw()

Compute the <u>ideal gas</u> mixture densities again at 200 atm. 

In [ ]:
# reset the rho array
rho[:] = 0.0e0
# loop over temperature data points
for i in range(len(T)):
    # set mixture temperature [K]
    premixed.temperature = T[i]
    # get mixture density [gm/cm3]
    rho[i] = premixed.RHO

Plot both the <u>ideal gas</u> and the <u>real-gas</u> density values at 200 atm for comparison.

In [ ]:
# reset the figure size
plt.plot(figsize=(6, 6))
# plot mixture density versus temperature
plt.plot(T, rho, curvelist[0])
plt.plot(T, rho_real, curvelist[2])
plt.ylabel("Density [g/cm3]")
plt.legend(("ideal gas", "real gas"), loc="upper right")
plt.show()

In [ ]:
# delete unused objects
del rho, rho_real, T, premixed

This tutorial example demonstrates the use of selected <strong>PyChemkin</strong> property utilities for mixtures. These tools allows you to build your own reactor simulations under the Python environment. Furthermore, it also describes how to switch between the <u>ideal gas law</u> and the <em>Chemkin</em> <u>real-gas cubic EOS model</u> in a <strong>PyChemkin</strong> project.